In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import plasma.functional as F

from plasma.meta.object_graph import Manager

/u02/thanh/workplace/plasma/.env/RnD/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class HUHU(F.AutoPipe):

    def __init__(self, name, age, test=5):
        super().__init__()
        self.name = name
        self.age = age
        self.test = test

In [4]:
class HEHE(F.AutoPipe):

    def __init__(self, huhu) -> None:
        super().__init__()
        self.huhu = huhu

In [5]:
class HAHA(F.AutoPipe):

    def __init__(self, hehe, huhu) -> None:
        super().__init__()
        self.hehe = hehe
        self.huhu = huhu

In [6]:
def decorator_provider(name, initiator):
    
    def initiate(*args, **kwargs):
        print(name)
        return initiator(*args, **kwargs)
    
    if name in {'huhu', 'hehe'}:
        return initiate
    else:
        return initiator        

In [7]:
injector = Manager()
injector\
    .add_dependency('huhu', HUHU)\
    .add_dependency('hehe', HEHE)\
    .add_dependency('hihi', HEHE)\
    .add_dependency('haha', HAHA)\
    .duplicate('huhu', 'new_huhu')\
    .add_decorator_provider(decorator_provider)

hihi
   |--huhu
      |--name
      |--age
      |--test=<class 'int'>
----------------------------------------------------------------------------------------------------
haha
   |--hehe
      |--huhu
         |--name
         |--age
         |--test=<class 'int'>
   |--huhu
      |--name
      |--age
      |--test=<class 'int'>
----------------------------------------------------------------------------------------------------
new_huhu
   |--name
   |--age
   |--test=<class 'int'>
----------------------------------------------------------------------------------------------------

In [8]:
objs = injector.run('haha', huhu=8)['haha']
objs

hehe


HAHA(
  hehe=HEHE(
    huhu=8,
  ),
  huhu=8,
)

In [9]:
injector

hihi
   |--huhu
      |--name
      |--age
      |--test=<class 'int'>
----------------------------------------------------------------------------------------------------
haha
   |--hehe
      |--huhu
         |--name
         |--age
         |--test=<class 'int'>
   |--huhu
      |--name
      |--age
      |--test=<class 'int'>
----------------------------------------------------------------------------------------------------
new_huhu
   |--name
   |--age
   |--test=<class 'int'>
----------------------------------------------------------------------------------------------------